In [2]:
import pandas as pd


df = pd.read_csv('/content/FInal.csv')
df.head()


,User ID,User Name,Driver Name,Car Condition,Weather,Traffic Condition,key,fare_amount,pickup_datetime,pickup_longitude,...,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing
0,KHVrEVlD,Kimberly Adams,Amy Butler,Very Good,windy,Congested Traffic,26:21.0,4.5,6/15/2009 17:26,-1.288826,...,6,0,2009,20.265840,55.176046,14.342611,34.543548,27.572573,1.030764,-2.918897
1,lPxIuEri,Justin Tapia,Hannah Zimmerman,Excellent,cloudy,Flow Traffic,52:16.0,16.9,1/5/2010 16:52,-1.291824,...,1,1,2010,44.667679,31.832358,23.130775,15.125872,8.755732,8.450134,-0.375217
2,gsVN8JLS,Elizabeth Lopez,Amanda Jackson,Bad,stormy,Congested Traffic,35:00.0,5.7,8/18/2011 0:35,-1.291242,...,8,3,2011,43.597686,33.712082,19.865289,17.722624,9.847344,1.389525,2.599961
3,9I7kWFgd,Steven Wilson,Amy Horn,Very Good,stormy,Flow Traffic,30:42.0,7.7,4/21/2012 4:30,-1.291319,...,4,5,2012,42.642965,32.556289,21.063132,15.738963,7.703421,2.799270,0.133905
4,8QN5ZaGN,Alexander Andrews,Cassandra Larson,Bad,stormy,Congested Traffic,51:00.0,5.3,3/9/2010 7:51,-1.290987,...,3,1,2010,43.329953,39.406828,15.219339,23.732406,15.600745,1.999157,-0.502703


In [3]:
from sklearn.preprocessing import LabelEncoder

columns_needed = [
    'fare_amount', 'distance', 'passenger_count', 'hour', 'day', 'month',
    'weekday', 'year', 'jfk_dist', 'ewr_dist', 'lga_dist', 'sol_dist',
    'nyc_dist', 'bearing', 'Car Condition', 'Weather', 'Traffic Condition'
]

df_model = df[columns_needed].dropna()

for col in ['Car Condition', 'Weather', 'Traffic Condition']:
    df_model[col] = LabelEncoder().fit_transform(df_model[col])


In [6]:
df_model.head()

,fare_amount,distance,passenger_count,hour,day,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,bearing,Car Condition,Weather,Traffic Condition
0,4.5,1.030764,1,17,15,6,0,2009,20.265840,55.176046,14.342611,34.543548,27.572573,-2.918897,3,4,0
1,16.9,8.450134,1,16,5,1,1,2010,44.667679,31.832358,23.130775,15.125872,8.755732,-0.375217,1,0,2
2,5.7,1.389525,2,0,18,8,3,2011,43.597686,33.712082,19.865289,17.722624,9.847344,2.599961,0,2,0
3,7.7,2.799270,1,4,21,4,5,2012,42.642965,32.556289,21.063132,15.738963,7.703421,0.133905,3,2,2
4,5.3,1.999157,1,7,9,3,1,2010,43.329953,39.406828,15.219339,23.732406,15.600745,-0.502703,0,2,0


In [5]:
from sklearn.model_selection import train_test_split

X = df_model.drop('fare_amount', axis=1)
y = df_model['fare_amount']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Model 1: Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_test)

# Model 2: Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)

# Model 3: XGBoost
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
xgb.fit(X_train, y_train)
xgb_preds = xgb.predict(X_test)

# Evaluation function
def evaluate_model(name, y_test, y_pred):
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"📊 {name} — R²: {r2:.4f}, RMSE: {rmse:.2f}")

# Evaluate all models
evaluate_model("Linear Regression", y_test, lr_preds)
evaluate_model("Random Forest", y_test, rf_preds)
evaluate_model("XGBoost", y_test, xgb_preds)


📊 Linear Regression — R²: 0.3506, RMSE: 8.02
📊 Random Forest — R²: 0.8008, RMSE: 4.44
📊 XGBoost — R²: 0.7931, RMSE: 4.53
